In [1292]:
#theme = 'plotly_dark'
theme = 'seaborn'

This k-means will find the min and max of data it's self and will quit once no points are to switch clusters and will find the "k" aka number of clusters by its self

It also plots the WCSS v. K  and plots the decrease amount v. K

In [1293]:
from math import floor
import plotly.express as px
import pandas as pd
import random
import centroid_2D as centroid_2D
import math
import plotly.graph_objects as go
import random


#genetrate input
data_random = []
data_cooked = []


N = 30
#random imput genarator size N  between 0 to 30
for i in range(N):
    x = random.randint(0, 30)
    y = random.randint(0, 30)
    cluster = 1
    data_random.append([x,y,cluster])

# NONrandom imput genarator size N  between 0 to 30
    
#this is for 4ish area example

p1 = floor(N/4) #first part of N
p2 = p1  #second part of N
p3 = p1 #third part
p4 = N - p1*3 # forth part 

for i in range(p1): 
    x = random.randint(2, 8)
    y = random.randint(1, 15)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p2): 
    x = random.randint(20, 29)
    y = random.randint(10, 20)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p3): 
    x = random.randint(14, 16)
    y = random.randint(16, 30)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(p4): 
    x = random.randint(0, 12)
    y = random.randint(25, 30)
    cluster = 1
    data_cooked.append([x,y, cluster])

'''
#this is for 2ish area example

h1 = floor(N/2) #first half of N
h2 = N-h1  #second half of N
for i in range(h1): 
    x = random.randint(15, 30)
    y = random.randint(10, 20)
    cluster = 1
    data_cooked.append([x,y, cluster])

for i in range(h2): 
    x = random.randint(0, 15)
    y = random.randint(0, 15)
    cluster = 1
    data_cooked.append([x,y, cluster])
'''

df_rand = pd.DataFrame(data_random)
df_cooked = pd.DataFrame(data_cooked)

df_rand =  df_rand.rename(columns={0:'X', 1:'Y', 2:'Cluster'})
df_cooked = df_cooked.rename(columns={0: 'X', 1: 'Y', 2:'Cluster'})

In [1294]:
#now plot Rando to see what it look like
#scatter_data = df_rand
scatter_data = df_cooked

fig_scatter = px.scatter(scatter_data, x='X', y='Y', color='Cluster')
fig_scatter.update_layout(template=theme)
fig_scatter.update_layout(width=800, height=800)
fig_scatter.show()


In [1295]:
def viz_data(data, centroids):
    data = pd.DataFrame(data)
    data = data.rename(columns={0: 'X', 1: 'Y', 2:'Cluster'})
    data["Cluster"] = data['Cluster'].astype(str)

    color_map = {'1': 'blue', '2': 'red','3': 'green','4': 'brown','5': 'orange','6': 'purple','7': 'pink','8': 'cyan','9': 'yellow','10': 'magenta'}

    fig = px.scatter(data, x='X', y='Y', color='Cluster', color_discrete_map=color_map, title="k-Means on Random Data")

    for c in centroids:
        num = str(c.cluster_number)
        color = color_map[num]
        fig.add_trace(go.Scatter(x=[c.x_old], y=[c.y_old], name="Centroid "+num, mode='markers', marker=dict(size=25, color=color)))
        fig.add_trace(go.Scatter(x=[c.x], y=[c.y], name="Next Spot for Centroid "+num, mode='markers', marker=dict(size=15, color='black', opacity=0.5)))

    fig.update_layout(template=theme)
    fig.update_layout(width=800, height=800)
    fig.show()

In [1296]:
k_to_WCSS_dic = {}

def wcss(c):
    sum_of_squares = 0
    if len(c.coordinates) == 0:
        return 0
    for point in c.coordinates:
        d = calc_distance(point[0]-c.x, point[1]-c.y)
        sum_of_squares += d*d
    return sum_of_squares
    

def unkown_means(data):
    for k in range(1, 31):
        centroids = k_means(k,data)
        #if k == 3:
        #viz_data(data=data,centroids=centroids)

        #now calc WCSS
        for c in centroids:
            k_to_WCSS_dic[k] = k_to_WCSS_dic.get(k, 0) + wcss(c)
        k_to_WCSS_dic[k] = k_to_WCSS_dic[k] / k
    
    print(k_to_WCSS_dic.keys())
    print(k_to_WCSS_dic.values())

    return centroids



def calc_distance(x,y):
    return math.sqrt((x*x) + (y*y))

def k_means(k, data):
    #create centriod
    centroids = [centroid_2D.Centroid() for _ in range(k)]

    
    #find the min and max in data
    x_min, x_max, y_min, y_max = 0,0,0,0
    for point in data:
        x_min = min(x_min, point[0])
        x_max = max(x_max, point[0])
        y_min = min(y_min, point[1])
        y_max = max(y_max, point[1])
    


    numbert = 1
    for c in centroids:
        c.cluster_number = numbert
        c.x = random.randint(x_min,x_max)
        c.y = random.randint(y_min,y_max)
        numbert += 1


    centroid_switched = True #if alreas on point switches centriod then this is true
    repetitions = 0 #we use this to make sure we are not stuck in infate loop of repetitions
    while centroid_switched and repetitions < 60:

        repetitions += 1
        centroid_switched = False

        #clearCentriods arrays
        for c in centroids:
            c.clear()  

        for point in data: #for each point find the centroid its closest to and add it to that 
            min_dist = float('inf')
            for c in centroids:
                new_dist = calc_distance(point[0]-c.x, point[1]-c.y)
                if new_dist < min_dist:
                    min_dist = new_dist
                    colosest_c = c       
            colosest_c.add_point(point[0], point[1])
            if colosest_c.cluster_number != point[2]: #this means it got to new centriod so we will wan to do at least one more iteration
                centroid_switched = True
            point[2] = colosest_c.cluster_number    #in the data change the centriod is assoside with

        for c in centroids:
            if len(c.coordinates) > 0:
                sum_x = 0
                sum_y = 0

                for point in c.coordinates:
                    sum_x += point[0]
                    sum_y += point[1]
                
                x_avg = sum_x/len(c.coordinates)
                y_avg = sum_y/len(c.coordinates)

                c.set_values(x_avg, y_avg)
    return centroids
    

In [1299]:
#viz_data(data=data_random,centroids=unkown_means(data_random))

unkown_means(data_random)

ss_df = pd.DataFrame.from_dict(k_to_WCSS_dic, orient='index', columns=['WCSS'])
ss_df.index.name = 'K'

fig_ss = px.line(ss_df, x=ss_df.index, y='WCSS', title='WCSS vs. K')
fig_ss.update_layout(template=theme)
fig_ss.show()

ss_df['Rate_of_Decrease'] = ss_df['WCSS'].diff() / ss_df.index.to_series().diff()

fig_rate_of_decrease = px.scatter(ss_df, x=ss_df.index, y='Rate_of_Decrease', title='Rate_of_Decrease vs. K')
fig_rate_of_decrease.update_layout(template=theme)
fig_rate_of_decrease.show()


dict_keys([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30])
dict_values([17130.000000000004, 2907.788497450262, 992.3970560303895, 435.00485061813185, 213.62968253968256, 197.72400793650797, 108.98301749271137, 94.150390625, 55.70084590763602, 52.78754761904762, 40.22749186075632, 27.2626350308642, 23.29483386436049, 20.40725218658892, 24.963345679012345, 20.754557291666668, 12.328957188411698, 10.150791609510746, 12.545708801088594, 8.3971875, 8.563950977216283, 9.958075068870523, 4.324347278156764, 4.454191984953704, 7.113136, 2.4703193749051735, 3.9140967670917375, 2.577790937803693, 1.902490193666544, 2.571308641975309])
